In [ ]:
import numpy as np
import pandas as pd
import json
import codecs
import datetime
import os
import sys

In [ ]:
sys.path.append('../')

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (24, 16)

In [ ]:
import oci
import json
from oci.config import validate_config
from sklearn.covariance import MinCovDet 

In [ ]:
namespace = "lrzustouvvrg"
bucket_name = "sdk-initial-data-bucket"
config = oci.config.from_file(
        "~/.oci/config",
        "DEFAULT")

In [ ]:
validate_config(config)

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
next_starts_with = None
object_list = object_storage_client.list_objects(namespace, bucket_name , fields='name, timeCreated, size', start=next_starts_with)

In [13]:
from vector_creator.score_vectors.score_vectors_assembly import score_vector_from_bucket


In [14]:
import multiprocessing as mp

In [15]:
res = pd.DataFrame()
for x in range (0,3):
    r = score_vector_from_bucket(object_storage_client,
                                 'photo-gallery', 
                                 namespace, 
                                 bucket_name, 
                                 next_starts_with,
                                 mp.cpu_count())
    next_starts_with = object_list.data.next_start_with
    res.append(r)

NameError: name 'mp' is not defined

In [ ]:
next_starts_with = object_list.data.next_start_with

In [ ]:
res1 = score_vector_from_bucket(object_storage_client, 'photo-gallery', next_starts_with)
#res1 = score_vector_from_bucket(object_storage_client, 'photo-gallery', namespace, bucket_name, next_starts_with)


In [ ]:
res = res0.append(res1)

In [ ]:
#res.to_csv('csv_files/photogallery_ver3.csv')

In [ ]:
res.index.is_unique

In [ ]:
inxs = res.index.values

In [ ]:
len(inxs)

In [ ]:
from sklearn import preprocessing
import ocifs
import oci 
from ocifs import OCIFileSystem

fs = OCIFileSystem()

In [ ]:
def norm_df_2(df):
    inxs = df.index.values
    std_scale = preprocessing.StandardScaler().fit(df)
    df_std = std_scale.transform(df)
    return pd.DataFrame(df_std, columns=df.columns, index=inxs)

#read the tags from json file for user register non register and create df with tag column
def create_tag_df(json_file_path, column_name): # 'entries.json'
    tag_data = json.load(codecs.open(json_file_path, 'r', 'utf-8-sig'))
    df_tag = pd.DataFrame(tag_data)
    df_tag = df_tag.rename(columns={column_name: ''}) # 'app_userid'
    dftag = df_tag.set_index('')
    return dftag

In [ ]:
df_n = norm_df_2(res)

In [ ]:
df_n.values[:,4].std()

In [ ]:
df_t = create_tag_df('entries.json', 'app_userid')
df_j = df_n.join(df_t)
df_j['User_Register'] = df_j['User_Register'].fillna('false')
df_j.head(5)

In [ ]:
len(df_j[df_j.User_Register == 'true'])

In [ ]:
#df_j.to_csv('csv_files/ModelFiles/photogallery_with_tag_for_xgboost_Oshik.csv')

In [ ]:
   # to write the data to object storage use oci://bucket#namespace/part_{i}.csv"
res1.to_csv('oci://photogallery_with_tag@lrzustouvvrg/photogallery_with_tag_for_xgboost_Oshik.csv')

In [ ]:
res1.head(5)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle
import pandas as pd

In [ ]:
from vector_creator.stats_models.estimators import *

In [ ]:
nz = z_score(res.values)

<a id='setup'></a>
 Setting up your notebook session to access ADB

In [ ]:
import os
# Enter connection information and wallet file directory

database_name = "DBlouie7"
wallet_storage_directory = os.path.join(os.path.expanduser("~"),"ADB")

In [ ]:
import ads
import configparser
import re
import shutil
from tempfile import NamedTemporaryFile
from zipfile import ZipFile

ads.set_documentation_mode(False)

# Create the wallet directory if missing: 
#os.makedirs(wallet_storage_directory, mode=0o700, exist_ok=True)

wallet_path = os.path.join(wallet_storage_directory, database_name)

# Prepare to store ADB connection information
adb_config = os.path.join(wallet_storage_directory, '.credentials')

# Write a configuration file for login creds.
config = configparser.ConfigParser()
config.read(adb_config)


### Test the connection to the ADB.

The easiest way to test the ADB connection is to run the `sqlplus` CLI tool. The `TNS_ADMIN` variable needs to be set and then the connection string is used to connect. This connection string is of the format `\<user_name>/\<password>@\<SID>`. This information can be obtained from the credentials file.

Once this is working, the notebook will demonstrate other methods to connect that are more practical for use within notebooks.

In [ ]:
# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = config[database_name].get('tns_admin')

# Test the database connection
creds = config[database_name]
connect = 'sqlplus ' + creds.get('user') + '/' + creds.get('password') + '@' + creds.get('sid')
print(os.popen(connect).read())

In [ ]:
connection_parameters = {
    "user_name": creds.get('user'),
    "password": creds.get('password'),
    "service_name": creds.get('sid'),
    "wallet_location": "/home/datascience/ADB/Wallet_DBlouie7.zip",
}

df_j.ads.to_sql(
    "photogallery_with_tag_for_xgboost",
    connection_parameters=connection_parameters,
    if_exists="replace"
)

In [ ]:
df_j.ads.to_sql(
    "photogallery_with_tag_for_xgboost",
    connection_parameters=connection_parameters,
    if_exists="replace"
)

In [ ]:
df_j.head(5)

for i, df in enumerate(pd.DataFrame.ads.read_sql(
        "SELECT * FROM SH.SALES",
        chunksize=100000 # rows per chunk,
        connection_parameters=connection_parameters,
      ))
   # each df will contain up to 100000 rows (chunksize)
   # to write the data to object storage use oci://bucket#namespace/part_{i}.csv"
   df_j.to_csv(f"(oci://photogallery_with_tag@lrzustouvvrg/part_{i}.csv")

In [ ]:
res1.to_csv("oci://photogallery_with_tag@myanmespace/photogallery_with_tags.csv", 
                storage_options={"config": "~/.oci/config"})